In [1]:
from keras import backend as K
from keras.engine.topology import Layer
import pickle
import tensorflow as tf
import keras.backend.tensorflow_backend as KTF
import os

#指定第一块GPU可用 
os.environ["CUDA_VISIBLE_DEVICES"] = '0'
config = tf.ConfigProto() 
#不全部占满显存, 按需分配
config.gpu_options.allow_growth=True
sess = tf.Session(config=config)

KTF.set_session(sess)

Using TensorFlow backend.


In [2]:
class Position_Embedding(Layer):
    
    def __init__(self, size=None, mode='sum', **kwargs):
        self.size = size #必须为偶数
        self.mode = mode
        super(Position_Embedding, self).__init__(**kwargs)
        
    def call(self, x):
        if (self.size == None) or (self.mode == 'sum'):
            self.size = int(x.shape[-1])
        batch_size,seq_len = K.shape(x)[0],K.shape(x)[1]
        position_j = 1. / K.pow(10000., \
                                 2 * K.arange(self.size / 2, dtype='float32' \
                               ) / self.size)
        position_j = K.expand_dims(position_j, 0)
        position_i = K.cumsum(K.ones_like(x[:,:,0]), 1)-1 #K.arange不支持变长，只好用这种方法生成
        position_i = K.expand_dims(position_i, 2)
        position_ij = K.dot(position_i, position_j)
        position_ij = K.concatenate([K.cos(position_ij), K.sin(position_ij)], 2)
        if self.mode == 'sum':
            return position_ij + x
        elif self.mode == 'concat':
            return K.concatenate([position_ij, x], 2)
        
    def compute_output_shape(self, input_shape):
        if self.mode == 'sum':
            return input_shape
        elif self.mode == 'concat':
            return (input_shape[0], input_shape[1], input_shape[2]+self.size)


class Attention(Layer):

    def __init__(self, nb_head, size_per_head, mask_right=False, **kwargs):
        self.nb_head = nb_head
        self.size_per_head = size_per_head
        self.output_dim = nb_head*size_per_head
        self.mask_right = mask_right
        super(Attention, self).__init__(**kwargs)

    def build(self, input_shape):
        self.WQ = self.add_weight(name='WQ', 
                                  shape=(input_shape[0][-1], self.output_dim),
                                  initializer='glorot_uniform',
                                  trainable=True)
        self.WK = self.add_weight(name='WK', 
                                  shape=(input_shape[1][-1], self.output_dim),
                                  initializer='glorot_uniform',
                                  trainable=True)
        self.WV = self.add_weight(name='WV', 
                                  shape=(input_shape[2][-1], self.output_dim),
                                  initializer='glorot_uniform',
                                  trainable=True)
        super(Attention, self).build(input_shape)
        
    def Mask(self, inputs, seq_len, mode='mul'):
        if seq_len == None:
            return inputs
        else:
            mask = K.one_hot(seq_len[:,0], K.shape(inputs)[1])
            mask = 1 - K.cumsum(mask, 1)
            for _ in range(len(inputs.shape)-2):
                mask = K.expand_dims(mask, 2)
            if mode == 'mul':
                return inputs * mask
            if mode == 'add':
                return inputs - (1 - mask) * 1e12
                
    def call(self, x):
        #如果只传入Q_seq,K_seq,V_seq，那么就不做Mask
        #如果同时传入Q_seq,K_seq,V_seq,Q_len,V_len，那么对多余部分做Mask
        if len(x) == 3:
            Q_seq,K_seq,V_seq = x
            Q_len,V_len = None,None
        elif len(x) == 5:
            Q_seq,K_seq,V_seq,Q_len,V_len = x
        #对Q、K、V做线性变换
        Q_seq = K.dot(Q_seq, self.WQ)
        Q_seq = K.reshape(Q_seq, (-1, K.shape(Q_seq)[1], self.nb_head, self.size_per_head))
        Q_seq = K.permute_dimensions(Q_seq, (0,2,1,3))
        K_seq = K.dot(K_seq, self.WK)
        K_seq = K.reshape(K_seq, (-1, K.shape(K_seq)[1], self.nb_head, self.size_per_head))
        K_seq = K.permute_dimensions(K_seq, (0,2,1,3))
        V_seq = K.dot(V_seq, self.WV)
        V_seq = K.reshape(V_seq, (-1, K.shape(V_seq)[1], self.nb_head, self.size_per_head))
        V_seq = K.permute_dimensions(V_seq, (0,2,1,3))
        #计算内积，然后mask，然后softmax
        A = K.batch_dot(Q_seq, K_seq, axes=[3,3]) / self.size_per_head**0.5
        A = K.permute_dimensions(A, (0,3,2,1))
        A = self.Mask(A, V_len, 'add')
        A = K.permute_dimensions(A, (0,3,2,1)) 
        if self.mask_right:
            ones = K.ones_like(A[:1, :1])
            mask = (ones - K.tf.matrix_band_part(ones, -1, 0)) * 1e12
            A = A - mask
        A = K.softmax(A)
        #输出并mask
        O_seq = K.batch_dot(A, V_seq, axes=[3,2])
        O_seq = K.permute_dimensions(O_seq, (0,2,1,3))
        O_seq = K.reshape(O_seq, (-1, K.shape(O_seq)[1], self.output_dim))
        O_seq = self.Mask(O_seq, Q_len, 'mul')
        return O_seq
        
    def compute_output_shape(self, input_shape):
        return (input_shape[0][0], input_shape[0][1], self.output_dim)


In [3]:
from keras.models import Model
from keras.layers import *
def attention_model():
    S_inputs = Input(shape=(None,), dtype='int32')
    embeddings = Embedding(max_features, 128)(S_inputs)
    embeddings = Position_Embedding()(embeddings) # 增加Position_Embedding能轻微提高准确率
    O_seq = Attention(8,16)([embeddings,embeddings,embeddings])
    O_seq = GlobalAveragePooling1D()(O_seq)
    O_seq = Dropout(0.5)(O_seq)
    outputs = Dense(1, activation='sigmoid')(O_seq)

    model = Model(inputs=S_inputs, outputs=outputs)
    # try using different optimizers and different optimizer configs
    model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
    return model

In [4]:
AA = 'GAVLIFWYDNEKQMSTCPHR'
def pep(path, seq_len):
    seqs = open(path).readlines()
    cut = (len(seqs[0].split()[0]) - 1 - seq_len) // 2
    X = [[AA.index(res.upper()) if res.upper() in AA else 0
          for res in (seq.split()[0][cut:-cut] if cut != 0 else seq.split()[0])]
        for seq in seqs if seq.strip() != '']
    y = [int(seq.split()[-1]) for seq in seqs if seq.strip() != '']
    return np.array(X), np.array(y)

In [ ]:
max_features = 20000
maxlen = 80
batch_size = 32

print('Loading data...')
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)
print(len(x_train), 'train sequences')
print(len(x_test), 'test sequences')

print('Pad sequences (samples x time)')
x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

from keras.models import Model
from keras.layers import *

S_inputs = Input(shape=(None,), dtype='int32')
embeddings = Embedding(max_features, 128)(S_inputs)
# embeddings = Position_Embedding()(embeddings) # 增加Position_Embedding能轻微提高准确率
O_seq = Attention(8,16)([embeddings,embeddings,embeddings])
O_seq = GlobalAveragePooling1D()(O_seq)
O_seq = Dropout(0.5)(O_seq)
outputs = Dense(1, activation='sigmoid')(O_seq)

model = Model(inputs=S_inputs, outputs=outputs)
# try using different optimizers and different optimizer configs
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

print('Train...')
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=5,
          validation_data=(x_test, y_test))

In [ ]:
from keras.preprocessing import sequence
from keras.datasets import imdb

max_features = 20000
maxlen = 80
batch_size = 32

print('Loading data...')
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)
print(len(x_train), 'train sequences')
print(len(x_test), 'test sequences')

print('Pad sequences (samples x time)')
x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

from keras.models import Model
from keras.layers import *

S_inputs = Input(shape=(None,), dtype='int32')
embeddings = Embedding(max_features, 128)(S_inputs)
# embeddings = Position_Embedding()(embeddings) # 增加Position_Embedding能轻微提高准确率
O_seq = Attention(8,16)([embeddings,embeddings,embeddings])
O_seq = GlobalAveragePooling1D()(O_seq)
O_seq = Dropout(0.5)(O_seq)
outputs = Dense(1, activation='sigmoid')(O_seq)

model = Model(inputs=S_inputs, outputs=outputs)
# try using different optimizers and different optimizer configs
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

print('Train...')
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=5,
          validation_data=(x_test, y_test))

In [5]:
import os,sys,re,time,math,csv


from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from sklearn import cluster
from sklearn.model_selection import train_test_split
from sklearn.metrics import *
from keras.callbacks import EarlyStopping



import matplotlib as mpl
import numpy as np
import pandas as pd

import sklearn
from matplotlib import pyplot as plt


from keras import backend as K

from keras.optimizers import Adam
from keras.models import *
from keras.layers import *
from keras.utils.np_utils import to_categorical


import gc
import tensorflow as tf
import warnings
warnings.filterwarnings('ignore')

from keras import backend as K
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import KFold
from keras.preprocessing import text, sequence
from keras.callbacks import Callback
from keras import optimizers
from keras import initializers, regularizers, constraints, callbacks


import pickle
from keras.preprocessing.sequence import pad_sequences
from keras_preprocessing.text import Tokenizer
import matplotlib.mlab as mlab
from keras import optimizers
from keras import regularizers
from keras.callbacks import TensorBoard, EarlyStopping, ModelCheckpoint
from keras.utils import to_categorical
from scipy import interp
from sklearn import metrics
from keras.models import load_model


In [25]:
from keras.engine.topology import Layer

class Position_Embedding(Layer):
    
    def __init__(self, size=None, mode='sum', **kwargs):
        self.size = size #必须为偶数
        self.mode = mode
        super(Position_Embedding, self).__init__(**kwargs)
        
    def call(self, x):
        if (self.size == None) or (self.mode == 'sum'):
            self.size = int(x.shape[-1])
        batch_size,seq_len = K.shape(x)[0],K.shape(x)[1]
        position_j = 1. / K.pow(10000., \
                                 2 * K.arange(self.size / 2, dtype='float32' \
                               ) / self.size)
        position_j = K.expand_dims(position_j, 0)
        position_i = K.cumsum(K.ones_like(x[:,:,0]), 1)-1 #K.arange不支持变长，只好用这种方法生成
        position_i = K.expand_dims(position_i, 2)
        position_ij = K.dot(position_i, position_j)
        position_ij = K.concatenate([K.cos(position_ij), K.sin(position_ij)], 2)
        if self.mode == 'sum':
            return position_ij + x
        elif self.mode == 'concat':
            return K.concatenate([position_ij, x], 2)
        
    def compute_output_shape(self, input_shape):
        if self.mode == 'sum':
            return input_shape
        elif self.mode == 'concat':
            return (input_shape[0], input_shape[1], input_shape[2]+self.size)


class Attention(Layer):

    def __init__(self, nb_head, size_per_head, mask_right=False, **kwargs):
        self.nb_head = nb_head
        self.size_per_head = size_per_head
        self.output_dim = nb_head*size_per_head
        self.mask_right = mask_right
        super(Attention, self).__init__(**kwargs)

    def build(self, input_shape):
        self.WQ = self.add_weight(name='WQ', 
                                  shape=(input_shape[0][-1], self.output_dim),
                                  initializer='glorot_uniform',
                                  trainable=True)
        self.WK = self.add_weight(name='WK', 
                                  shape=(input_shape[1][-1], self.output_dim),
                                  initializer='glorot_uniform',
                                  trainable=True)
        self.WV = self.add_weight(name='WV', 
                                  shape=(input_shape[2][-1], self.output_dim),
                                  initializer='glorot_uniform',
                                  trainable=True)
        super(Attention, self).build(input_shape)
        
    def Mask(self, inputs, seq_len, mode='mul'):
        if seq_len == None:
            return inputs
        else:
            mask = K.one_hot(seq_len[:,0], K.shape(inputs)[1])
            mask = 1 - K.cumsum(mask, 1)
            for _ in range(len(inputs.shape)-2):
                mask = K.expand_dims(mask, 2)
            if mode == 'mul':
                return inputs * mask
            if mode == 'add':
                return inputs - (1 - mask) * 1e12
                
    def call(self, x):
        #如果只传入Q_seq,K_seq,V_seq，那么就不做Mask
        #如果同时传入Q_seq,K_seq,V_seq,Q_len,V_len，那么对多余部分做Mask
        if len(x) == 3:
            Q_seq,K_seq,V_seq = x
            Q_len,V_len = None,None
        elif len(x) == 5:
            Q_seq,K_seq,V_seq,Q_len,V_len = x
        #对Q、K、V做线性变换
        Q_seq = K.dot(Q_seq, self.WQ)
        Q_seq = K.reshape(Q_seq, (-1, K.shape(Q_seq)[1], self.nb_head, self.size_per_head))
        Q_seq = K.permute_dimensions(Q_seq, (0,2,1,3))
        K_seq = K.dot(K_seq, self.WK)
        K_seq = K.reshape(K_seq, (-1, K.shape(K_seq)[1], self.nb_head, self.size_per_head))
        K_seq = K.permute_dimensions(K_seq, (0,2,1,3))
        V_seq = K.dot(V_seq, self.WV)
        V_seq = K.reshape(V_seq, (-1, K.shape(V_seq)[1], self.nb_head, self.size_per_head))
        V_seq = K.permute_dimensions(V_seq, (0,2,1,3))
        #计算内积，然后mask，然后softmax
        A = K.batch_dot(Q_seq, K_seq, axes=[3,3]) / self.size_per_head**0.5
        A = K.permute_dimensions(A, (0,3,2,1))
        A = self.Mask(A, V_len, 'add')
        A = K.permute_dimensions(A, (0,3,2,1)) 
        if self.mask_right:
            ones = K.ones_like(A[:1, :1])
            mask = (ones - K.tf.matrix_band_part(ones, -1, 0)) * 1e12
            A = A - mask
        A = K.softmax(A)
        #输出并mask
        O_seq = K.batch_dot(A, V_seq, axes=[3,2])
        O_seq = K.permute_dimensions(O_seq, (0,2,1,3))
        O_seq = K.reshape(O_seq, (-1, K.shape(O_seq)[1], self.output_dim))
        O_seq = self.Mask(O_seq, Q_len, 'mul')
        return O_seq
        
    def compute_output_shape(self, input_shape):
        return (input_shape[0][0], input_shape[0][1], self.output_dim)


In [6]:
def create_cnn_model6(input_length=29,dropout=0.5):
    model = Sequential()
    model.add(Embedding(21, 5, input_length = input_length))
    model.add(Conv1D(128, 8, activation='relu', padding='same'))
    #model.add(MaxPooling1D(2))
    model.add(Dropout(dropout))

    model.add(Conv1D(128, 8, activation='relu', padding='same'))
    #model.add(MaxPooling1D(2))
    model.add(Dropout(dropout))

    model.add(Conv1D(128, 8, activation='relu', padding='same'))
    model.add(MaxPooling1D(2))
    model.add(Dropout(dropout))

    model.add(Flatten())
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
    return model

In [7]:
def mean(a):
    return sum(a) / len(a)

In [8]:
def DPCNN1():
    filter_nr = 128 #滤波器通道个数
    filter_size = 3 #卷积核
    max_pool_size = 3 #池化层的pooling_size
    max_pool_strides = 2 #池化层的步长
    dense_nr = 256 #全连接层
    spatial_dropout = 0.2
    dense_dropout = 0.5
    train_embed = False
    conv_kern_reg = regularizers.l2(0.00001)
    conv_bias_reg = regularizers.l2(0.00001)

    comment = Input(shape=(maxlen,))
#     emb_comment = Embedding(max_features, embed_size, weights=[embedding_matrix], trainable=train_embed)(comment)
    emb_comment = Embedding(max_features, embed_size, trainable=train_embed)(comment)
    emb_comment = SpatialDropout1D(spatial_dropout)(emb_comment)

    block1 = Conv1D(filter_nr, kernel_size=filter_size, padding='same', activation='linear', 
                kernel_regularizer=conv_kern_reg, bias_regularizer=conv_bias_reg)(emb_comment)
    block1 = BatchNormalization()(block1)
    block1 = PReLU()(block1)
    block1 = Conv1D(filter_nr, kernel_size=filter_size, padding='same', activation='linear', 
                kernel_regularizer=conv_kern_reg, bias_regularizer=conv_bias_reg)(block1)
    block1 = BatchNormalization()(block1)
    block1 = PReLU()(block1)

    #we pass embedded comment through conv1d with filter size 1 because it needs to have the same shape as block output
    #if you choose filter_nr = embed_size (300 in this case) you don't have to do this part and can add emb_comment directly to block1_output
    resize_emb = Conv1D(filter_nr, kernel_size=1, padding='same', activation='linear', 
                kernel_regularizer=conv_kern_reg, bias_regularizer=conv_bias_reg)(emb_comment)
    resize_emb = PReLU()(resize_emb)

    
    
    block1_output = add([block1, resize_emb])
    block1_output = MaxPooling1D(pool_size=max_pool_size, strides=max_pool_strides)(block1_output)

    block2 = Conv1D(filter_nr, kernel_size=filter_size, padding='same', activation='linear', 
                kernel_regularizer=conv_kern_reg, bias_regularizer=conv_bias_reg)(block1_output)
    block2 = BatchNormalization()(block2)
    block2 = PReLU()(block2)
    block2 = Conv1D(filter_nr, kernel_size=filter_size, padding='same', activation='linear', 
                kernel_regularizer=conv_kern_reg, bias_regularizer=conv_bias_reg)(block2)
    block2 = BatchNormalization()(block2)
    block2 = PReLU()(block2)

    block2_output = add([block2, block1_output])
    block2_output = MaxPooling1D(pool_size=max_pool_size, strides=max_pool_strides)(block2_output)

    
    
    block3 = Conv1D(filter_nr, kernel_size=filter_size, padding='same', activation='linear', 
                kernel_regularizer=conv_kern_reg, bias_regularizer=conv_bias_reg)(block2_output)
    block3 = BatchNormalization()(block3)
    block3 = PReLU()(block3)
    block3 = Conv1D(filter_nr, kernel_size=filter_size, padding='same', activation='linear', 
                kernel_regularizer=conv_kern_reg, bias_regularizer=conv_bias_reg)(block3)
    block3 = BatchNormalization()(block3)
    block3 = PReLU()(block3)

    block3_output = add([block3, block2_output])
    block3_output = MaxPooling1D(pool_size=max_pool_size, strides=max_pool_strides)(block3_output)

    
    block4 = Conv1D(filter_nr, kernel_size=filter_size, padding='same', activation='linear', 
                kernel_regularizer=conv_kern_reg, bias_regularizer=conv_bias_reg)(block3_output)
    block4 = BatchNormalization()(block4)
    block4 = PReLU()(block4)
    block4 = Conv1D(filter_nr, kernel_size=filter_size, padding='same', activation='linear', 
                kernel_regularizer=conv_kern_reg, bias_regularizer=conv_bias_reg)(block4)
    block4 = BatchNormalization()(block4)
    block4 = PReLU()(block4)

    block4_output = add([block4, block3_output])
    block4_output = MaxPooling1D(pool_size=max_pool_size, strides=max_pool_strides)(block4_output)
    
    
    block5 = Conv1D(filter_nr, kernel_size=filter_size, padding='same', activation='linear', 
                kernel_regularizer=conv_kern_reg, bias_regularizer=conv_bias_reg)(block4_output)
    block5 = BatchNormalization()(block5)
    block5 = PReLU()(block5)
    block5 = Conv1D(filter_nr, kernel_size=filter_size, padding='same', activation='linear', 
                kernel_regularizer=conv_kern_reg, bias_regularizer=conv_bias_reg)(block5)
    block5 = BatchNormalization()(block5)
    block5 = PReLU()(block5)

    block5_output = add([block5, block4_output])
    block5_output = MaxPooling1D(pool_size=max_pool_size, strides=max_pool_strides)(block5_output)

    block6 = Conv1D(filter_nr, kernel_size=filter_size, padding='same', activation='linear', 
                kernel_regularizer=conv_kern_reg, bias_regularizer=conv_bias_reg)(block5_output)
    block6 = BatchNormalization()(block6)
    block6 = PReLU()(block6)
    block6 = Conv1D(filter_nr, kernel_size=filter_size, padding='same', activation='linear', 
                kernel_regularizer=conv_kern_reg, bias_regularizer=conv_bias_reg)(block6)
    block6 = BatchNormalization()(block6)
    block6 = PReLU()(block6)

    block6_output = add([block6, block5_output])
    block6_output = MaxPooling1D(pool_size=max_pool_size, strides=max_pool_strides)(block6_output)

    block7 = Conv1D(filter_nr, kernel_size=filter_size, padding='same', activation='linear', 
                kernel_regularizer=conv_kern_reg, bias_regularizer=conv_bias_reg)(block6_output)
    block7 = BatchNormalization()(block7)
    block7 = PReLU()(block7)
    block7 = Conv1D(filter_nr, kernel_size=filter_size, padding='same', activation='linear', 
                kernel_regularizer=conv_kern_reg, bias_regularizer=conv_bias_reg)(block7)
    block7 = BatchNormalization()(block7)
    block7 = PReLU()(block7)

    block7_output = add([block7, block6_output])
    output = GlobalMaxPooling1D()(block7_output)

    output = Dense(dense_nr, activation='linear')(output)
    output = BatchNormalization()(output)
    output = PReLU()(output)
    output = Dropout(dense_dropout)(output)
    output = Dense(8, activation='softmax')(output)

    model = Model(comment, output)
    model.summary()
    model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
    return model

In [9]:
def CNN(x):
    
    block = Conv1D(filter_nr, kernel_size=filter_size, padding='same', activation='linear', 
                kernel_regularizer=conv_kern_reg, bias_regularizer=conv_bias_reg)(x)
    block = BatchNormalization()(block)
    block = PReLU()(block)
    block = Conv1D(filter_nr, kernel_size=filter_size, padding='same', activation='linear', 
                kernel_regularizer=conv_kern_reg, bias_regularizer=conv_bias_reg)(block)
    block = BatchNormalization()(block)
    block = PReLU()(block)
    return block




filter_nr = 64 #滤波器通道个数
filter_size = 3 #卷积核
max_pool_size = 3 #池化层的pooling_size
max_pool_strides = 2 #池化层的步长
dense_nr = 256 #全连接层
spatial_dropout = 0.2
dense_dropout = 0.5
train_embed = False
conv_kern_reg = regularizers.l2(0.00001)
conv_bias_reg = regularizers.l2(0.00001)


filter_nr = 128
filter_size = 3
max_pool_size = 3
max_pool_strides = 2
dense_nr = 256
spatial_dropout = 0.4
dense_dropout = 0.4
train_embed = False
    
def DPCNN2():
    comment = Input(shape=(maxlen,))
#     emb_comment = Embedding(max_features, embed_size, weights=[embedding_matrix], trainable=train_embed)(comment)
    emb_comment = Embedding(max_features, embed_size,trainable=train_embed)(comment)
    emb_comment = SpatialDropout1D(spatial_dropout)(emb_comment)

    #region embedding层
    resize_emb = Conv1D(filter_nr, kernel_size=1,padding='same', activation='linear',  
                kernel_regularizer=conv_kern_reg, bias_regularizer=conv_bias_reg)(emb_comment)
    resize_emb = PReLU()(resize_emb)
    #第一层
    block1 = CNN(emb_comment)
    block1_output = add([block1, resize_emb])
    block1_output = MaxPooling1D(pool_size=max_pool_size, strides=max_pool_strides)(block1_output)
    #第二层
    block2 = CNN(block1_output)
    block2_output = add([block2, block1_output])
    block2_output = MaxPooling1D(pool_size=max_pool_size, strides=max_pool_strides)(block2_output)
    #第三层
    block3 = CNN(block2_output)
    block3_output = add([block3, block2_output])
    block3_output = MaxPooling1D(pool_size=max_pool_size, strides=max_pool_strides)(block3_output)  
    #第四层
    block4 = CNN(block3_output) 
    block4_output = add([block4, block3_output])
    block4_output = MaxPooling1D(pool_size=max_pool_size, strides=max_pool_strides)(block4_output)
    #第五层
    block5 = CNN(block4_output) 
    block5_output = add([block5, block4_output])
    block5_output = MaxPooling1D(pool_size=max_pool_size, strides=max_pool_strides)(block5_output)
    #第六层
    block6 = CNN(block5_output) 
    block6_output = add([block6, block5_output])
    block6_output = MaxPooling1D(pool_size=max_pool_size, strides=max_pool_strides)(block6_output)
    #第七层
    block7 = CNN(block6_output) 
    block7_output = add([block7, block6_output])
    output = GlobalMaxPooling1D()(block7_output)
    #全连接层
    output = Dense(dense_nr, activation="linear")(output)
    output = BatchNormalization()(output)
    output = PReLU()(output)
    output = Dropout(dense_dropout)(output)
    output = Dense(8, activation="softmax")(output)

    model = Model(comment, output)
    model.summary()
    model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
    return model

In [10]:
def DPCNN3():
    project_maxlen = 1000
    
    filter_nr = 128
    filter_size = 3
    max_pool_size = 3
    max_pool_strides = 2
    dense_nr = 256
    spatial_dropout = 0.4
    dense_dropout = 0.3
    train_embed = False
    
    pj_repeat = 5     # dpcnn block repeated times on project text 

    project = Input(shape=(project_maxlen,), name='project')
    emb_project = Embedding(max_features, embed_size, trainable=train_embed)(project)
#     emb_project = Embedding(304, 256)(project)
    emb_project = SpatialDropout1D(spatial_dropout)(emb_project)
    pj_block1 = Conv1D(filter_nr, kernel_size=filter_size, padding='same', activation='linear')(emb_project)
    pj_block1 = BatchNormalization()(pj_block1)
    pj_block1 = PReLU()(pj_block1)
    pj_block1 = Conv1D(filter_nr, kernel_size=filter_size, padding='same', activation='linear')(pj_block1)
    pj_block1 = BatchNormalization()(pj_block1)
    pj_block1 = PReLU()(pj_block1)
    pj_resize_emb = Conv1D(filter_nr, kernel_size=1, padding='same', activation='linear')(emb_project)
    pj_resize_emb = PReLU()(pj_resize_emb)
    pj_block1_output = add([pj_block1, pj_resize_emb])
    for _ in range(pj_repeat):  
        pj_block1_output = MaxPooling1D(pool_size=max_pool_size, strides=max_pool_strides)(pj_block1_output)
        pj_block2 = Conv1D(filter_nr, kernel_size=filter_size, padding='same', activation='linear')(pj_block1_output)
        pj_block2 = BatchNormalization()(pj_block2)
        pj_block2 = PReLU()(pj_block2)
        pj_block2 = Conv1D(filter_nr, kernel_size=filter_size, padding='same', activation='linear')(pj_block2)
        pj_block2 = BatchNormalization()(pj_block2)
        pj_block2 = PReLU()(pj_block2)
        pj_block1_output = add([pj_block2, pj_block1_output])
    
        
    pj_output = GlobalMaxPooling1D()(pj_block1_output)
    pj_output = BatchNormalization()(pj_output)
    
    
    output = Dense(dense_nr, activation='linear')(pj_output)
    output = BatchNormalization()(output)
    output = PReLU()(output)
    output = Dropout(dense_dropout)(output)
    output = Dense(8, activation='softmax')(output)
    model = Model(inputs=project, outputs=output)
#     model.compile(loss='binary_crossentropy', 
#                 optimizer='adam',
#                 metrics=['accuracy'])
    model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
#     model.summary()
    return model

In [11]:
# project  resources  train_seq
# 2st DPCNN + Pseudo-Labelling
def get_model2():
#     session_conf = tf.ConfigProto(intra_op_parallelism_threads=4, inter_op_parallelism_threads=4)
#     K.set_session(tf.Session(graph=tf.get_default_graph(), config=session_conf))

    filter_nr = 128
    filter_size = 3
    max_pool_size = 3
    max_pool_strides = 2
    dense_nr = 256
    spatial_dropout = 0.4
    dense_dropout = 0.3
    train_embed = False
    
    pj_repeat = 5     # dpcnn block repeated times on project text 
    rs_repeat = 1     # dpcnn block repeated times on resources text 

    project = Input(shape=(project_maxlen,), name='project')
#     emb_project = Embedding(max_features, embed_size, weights=[embedding_matrix], trainable=train_embed)(project)
    emb_project = Embedding(max_features, embed_size, trainable=train_embed)(project)
    emb_project = SpatialDropout1D(spatial_dropout)(emb_project)
    pj_block1 = Conv1D(filter_nr, kernel_size=filter_size, padding='same', activation='linear')(emb_project)
    pj_block1 = BatchNormalization()(pj_block1)
    pj_block1 = PReLU()(pj_block1)
    pj_block1 = Conv1D(filter_nr, kernel_size=filter_size, padding='same', activation='linear')(pj_block1)
    pj_block1 = BatchNormalization()(pj_block1)
    pj_block1 = PReLU()(pj_block1)
    pj_resize_emb = Conv1D(filter_nr, kernel_size=1, padding='same', activation='linear')(emb_project)
    pj_resize_emb = PReLU()(pj_resize_emb)
    pj_block1_output = add([pj_block1, pj_resize_emb])
    for _ in range(pj_repeat):  
        pj_block1_output = MaxPooling1D(pool_size=max_pool_size, strides=max_pool_strides)(pj_block1_output)
        pj_block2 = Conv1D(filter_nr, kernel_size=filter_size, padding='same', activation='linear')(pj_block1_output)
        pj_block2 = BatchNormalization()(pj_block2)
        pj_block2 = PReLU()(pj_block2)
        pj_block2 = Conv1D(filter_nr, kernel_size=filter_size, padding='same', activation='linear')(pj_block2)
        pj_block2 = BatchNormalization()(pj_block2)
        pj_block2 = PReLU()(pj_block2)
        pj_block1_output = add([pj_block2, pj_block1_output])
    
    resouse = Input(shape=(resouse_max_len,), name='resouse')
#     emb_resouse = Embedding(max_features, embed_size, weights=[embedding_matrix], trainable=train_embed)(resouse)
    emb_project = Embedding(max_features, embed_size, trainable=train_embed)(project)
    emb_resouse = SpatialDropout1D(spatial_dropout)(emb_resouse)
    rs_block1 = Conv1D(filter_nr, kernel_size=filter_size, padding='same', activation='linear')(emb_resouse)
    rs_block1 = BatchNormalization()(rs_block1)
    rs_block1 = PReLU()(rs_block1)
    rs_block1 = Conv1D(filter_nr, kernel_size=filter_size, padding='same', activation='linear')(rs_block1)
    rs_block1 = BatchNormalization()(rs_block1)
    rs_block1 = PReLU()(rs_block1)
    rs_resize_emb = Conv1D(filter_nr, kernel_size=1, padding='same', activation='linear')(emb_resouse)
    rs_resize_emb = PReLU()(rs_resize_emb)

    rs_block1_output = add([rs_block1, rs_resize_emb])
    for _ in range(rs_repeat):  
        rs_block1_output = MaxPooling1D(pool_size=max_pool_size, strides=max_pool_strides)(rs_block1_output)
        rs_block2 = Conv1D(filter_nr, kernel_size=filter_size, padding='same', activation='linear')(rs_block1_output)
        rs_block2 = BatchNormalization()(rs_block2)
        rs_block2 = PReLU()(rs_block2)
        rs_block2 = Conv1D(filter_nr, kernel_size=filter_size, padding='same', activation='linear')(rs_block2)
        rs_block2 = BatchNormalization()(rs_block2)
        rs_block2 = PReLU()(rs_block2)
        rs_block1_output = add([rs_block2, rs_block1_output])
        
    pj_output = GlobalMaxPooling1D()(pj_block1_output)
    rs_output = GlobalMaxPooling1D()(rs_block1_output)
    pj_output = BatchNormalization()(pj_output)
    rs_output = BatchNormalization()(rs_output)
    
    inp_num = Input(shape=(train_seq.shape[1]-maxlen, ), name='num_input')
    conc = concatenate([pj_output, rs_output, inp_num])
    
    output = Dense(dense_nr, activation='linear')(conc)
    output = BatchNormalization()(output)
    output = PReLU()(output)
    output = Dropout(dense_dropout)(output)
    output = Dense(1, activation='sigmoid')(output)
    
    
    model = Model(inputs=[project, resouse, inp_num], outputs=output)
#     model.compile(loss='binary_crossentropy', 
#                 optimizer='adam',
#                 metrics=['accuracy'])
    model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
    model.summary()
    return model

In [26]:
def attention_model2():
    S_inputs = Input(shape=(None,), dtype='int32')
    embeddings = Embedding(304, 256)(S_inputs)
    embeddings = Position_Embedding()(embeddings) # 增加Position_Embedding能轻微提高准确率
    O_seq = Attention(8,16)([embeddings,embeddings,embeddings])
    O_seq = GlobalAveragePooling1D()(O_seq)
    O_seq = Dropout(0.5)(O_seq)
    outputs = Dense(8, activation='softmax')(O_seq)

    model = Model(inputs=S_inputs, outputs=outputs)
    # try using different optimizers and different optimizer configs
    # model.compile(loss='binary_crossentropy',
    #               optimizer='adam',
    #               metrics=['accuracy'])
    model.compile(loss='categorical_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])
    return model

In [ ]:
@contextmanager
def timer(title):
    t0 = time.time()
    yield
    print("{} - done in {:.2f}s".format(title, time.time() - t0))
   

In [34]:
# 重新训练CNN word Embedding
max_features = 29
name = 'Embedding'
gap = ''
auc_mean=[]
path_train = 'C:/Users/Crow/Desktop/human_data_12.12/Train.txt'
path_test =  'C:/Users/Crow/Desktop/human_data_12.12/Independent.txt'

x_train,y_train = pep(path_train,29-2)
x_test,y_test = pep(path_test,29-2)

kf = KFold(n_splits = 10,random_state=5,shuffle=True)
j = 1 
for train_index, test_index in kf.split(x_train):
    x_train3, x_test3 = x_train[train_index], x_train[test_index]
    y_train3, y_test3 = y_train[train_index], y_train[test_index]
    
#     model = create_cnn_model5(input_length=29)
    
#     early_stopping = EarlyStopping(monitor='val_loss', patience=5)
#     callbacks_list = [early_stopping]
#     model.fit(x_train3, y_train3, validation_data = (x_test3, y_test3), epochs = 20, batch_size = 256,shuffle=True,
#          callbacks=callbacks_list, verbose=1)
    model = create_cnn_model6()
    
    #filepath='C:/Users/Crow/Desktop/new_result/CNN6/model/29_kfold_CNN_'+ name + gap+'_'+ str(j) +'.hdf5'
   
    #filepath="C:/Users/Crow/Desktop/result/re_CNN/model/weights.best.hdf5"
    #checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=False,mode='auto', period=50)
    
    early_stopping = EarlyStopping(monitor='val_loss', patience=3)
    callbacks_list = [early_stopping]
    model.fit(x_train3, y_train3, validation_data = (x_test3, y_test3), epochs = 200, batch_size = 256,
              shuffle=True,callbacks=callbacks_list, verbose=1)
    
    
    
    
    test_pred_proba = model.predict(x_test3)
    fpr, tpr, thresholds = roc_curve(y_test3,test_pred_proba,pos_label=1)
    #print("ACC:  %f "  %accuracy_score(y_test3,test_pred))
    print("AUC: %f" % auc(fpr, tpr))
    auc_mean.append(auc(fpr, tpr))
    #print("MCC: %f " %matthews_corrcoef(y_test3,test_pred))
#     fw = open('C:/Users/Crow/Desktop/new_result/CNN6/29_kfold_CNN_'+ name + gap+'_result_'+ str(j) +'.txt','w')
#     for t in range(0,len(test_pred_proba)):
#         fw.write(str(test_pred_proba[t][0]))
#         fw.write('\t')
#         fw.write(str(y_test3[t]))
#         fw.write('\n')
#     fw.close()
    
#     fw = open('C:/Users/Crow/Desktop/new_result/CNN6/29_kfold_CNN_'+ name + gap+'_test_'+ str(j) +'.txt','w')
#     for t in range(0,len(y_test3)):
#         fw.write(str(y_test3[t]))
#         fw.write('\n')
#     fw.close()
    
    if j == 10:
        print(auc_mean)
        print(print("CV AUC: %f" % mean(auc_mean)))
#         model.save('C:/Users/Crow/Desktop/result/re_CNN/model/CNN_kfold_'+ name + gap +'.h5') 
        
#         test_pred_proba = model.predict(x_test)
#         fpr, tpr, thresholds = roc_curve(y_test,test_pred_proba,pos_label=1)
#         print("总AUC: %f" % auc(fpr, tpr))
#         fw = open('C:/Users/Crow/Desktop/result/re_CNN/29_kfold_CNN_'+ name + gap +'_result.txt','w')
#         for t in range(0,len(test_pred_proba)):
#             fw.write(str(test_pred_proba[t][0]))
#             fw.write('\t')
#             fw.write(str(y_test[t]))
#             fw.write('\n') 
#         fw.close()
    j+=1

Train on 66413 samples, validate on 7380 samples
Epoch 1/200
66413/66413 [==============================] - 48s 723us/step - loss: 0.3023 - acc: 0.9084 - val_loss: 0.2735 - val_acc: 0.9077
Epoch 2/200
66413/66413 [==============================] - 2s 37us/step - loss: 0.2644 - acc: 0.9096 - val_loss: 0.2610 - val_acc: 0.9077
Epoch 3/200
66413/66413 [==============================] - 3s 38us/step - loss: 0.2539 - acc: 0.9096 - val_loss: 0.2484 - val_acc: 0.9077
Epoch 4/200
66413/66413 [==============================] - 2s 37us/step - loss: 0.2464 - acc: 0.9096 - val_loss: 0.2480 - val_acc: 0.9077
Epoch 5/200
66413/66413 [==============================] - 2s 38us/step - loss: 0.2413 - acc: 0.9093 - val_loss: 0.2418 - val_acc: 0.9077
Epoch 6/200
66413/66413 [==============================] - 2s 38us/step - loss: 0.2388 - acc: 0.9094 - val_loss: 0.2377 - val_acc: 0.9080
Epoch 7/200
66413/66413 [==============================] - 2s 37us/step - loss: 0.2371 - acc: 0.9096 - val_loss: 0.2362 -

66414/66414 [==============================] - 3s 39us/step - loss: 0.2296 - acc: 0.9098 - val_loss: 0.2228 - val_acc: 0.9091
Epoch 10/200
66414/66414 [==============================] - 3s 39us/step - loss: 0.2276 - acc: 0.9099 - val_loss: 0.2246 - val_acc: 0.9088
Epoch 11/200
66414/66414 [==============================] - 3s 39us/step - loss: 0.2268 - acc: 0.9104 - val_loss: 0.2413 - val_acc: 0.9024
Epoch 12/200
66414/66414 [==============================] - 3s 38us/step - loss: 0.2266 - acc: 0.9105 - val_loss: 0.2209 - val_acc: 0.9089
Epoch 13/200
66414/66414 [==============================] - 3s 38us/step - loss: 0.2235 - acc: 0.9103 - val_loss: 0.2319 - val_acc: 0.9089
Epoch 14/200
66414/66414 [==============================] - 3s 38us/step - loss: 0.2223 - acc: 0.9105 - val_loss: 0.2247 - val_acc: 0.9078
Epoch 15/200
66414/66414 [==============================] - 3s 38us/step - loss: 0.2211 - acc: 0.9112 - val_loss: 0.2204 - val_acc: 0.9084
Epoch 16/200
66414/66414 [==============

In [18]:
max_features = 304
embed_size = 256
model = DPCNN1()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            (None, 1000)         0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 1000, 256)    77824       input_4[0][0]                    
__________________________________________________________________________________________________
spatial_dropout1d_1 (SpatialDro (None, 1000, 256)    0           embedding_1[0][0]                
__________________________________________________________________________________________________
conv1d_1 (Conv1D)               (None, 1000, 64)     49216       spatial_dropout1d_1[0][0]        
__________________________________________________________________________________________________
batch_norm

In [12]:
# 构建词向量


with open("security_test.csv.pkl", "rb") as f:
    file_names = pickle.load(f)
    outfiles = pickle.load(f)
with open("security_train.csv.pkl", "rb") as f:
    labels_d = pickle.load(f)
with open("security_train.csv.pkl", "rb") as f:
    labels = pickle.load(f)
    files = pickle.load(f)


labels = np.asarray(labels)
labels = to_categorical(labels, num_classes=8)

tokenizer = Tokenizer(num_words=None,
                      filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~',
                      split=' ',
                      char_level=False,
                      oov_token=None)
tokenizer.fit_on_texts(files)
tokenizer.fit_on_texts(outfiles)

# with open("wordsdic.pkl", 'wb') as f:
#     pickle.dump(tokenizer, f)

vocab = tokenizer.word_index
#print(tokenizer.word_index)
print(len(vocab))


x_train_word_ids = tokenizer.texts_to_sequences(files)
x_out_word_ids = tokenizer.texts_to_sequences(outfiles)

303


In [28]:
"""
    DPCNN 3st: 
        filter_nr = 128 #滤波器通道个数
        filter_size = 3 #卷积核
        max_pool_size = 3 #池化层的pooling_size
        max_pool_strides = 2 #池化层的步长
        dense_nr = 256 #全连接层
        spatial_dropout = 0.2
        dense_dropout = 0.5
        train_embed = False
        conv_kern_reg = regularizers.l2(0.00001)
        conv_bias_reg = regularizers.l2(0.00001)
"""
maxlen = 1000


x_train_padded_seqs = pad_sequences(x_train_word_ids, maxlen=maxlen)
x_out_padded_seqs = pad_sequences(x_out_word_ids, maxlen=maxlen)


meta_train = np.zeros(shape=(len(x_train_padded_seqs), 8))
meta_test = np.zeros(shape=(len(x_out_padded_seqs), 8))


Fname = 'malware_attention_'
Time = Fname + str(time.strftime("%Y-%m-%d-%H-%M-%S", time.localtime()))
tensorboard = TensorBoard(log_dir='./Logs/' + Time, histogram_freq=0, write_graph=True, write_images=False,
                          embeddings_freq=0, embeddings_layer_names=None, embeddings_metadata=None)

skf = StratifiedKFold(n_splits=5, random_state=4, shuffle=True)
for i, (tr_ind, te_ind) in enumerate(skf.split(x_train_padded_seqs, labels_d)):
    print('FOLD: {}'.format(str(i)))
    print(len(te_ind), len(tr_ind))
    X_train, X_train_label = x_train_padded_seqs[tr_ind], labels[tr_ind]
    X_val, X_val_label = x_train_padded_seqs[te_ind], labels[te_ind]

    #model = dila()
    
    # 先测试TextCNN模型
    #model = attention_model2()
    model = attention_model2()

    # model = load_model('model_weight.h5')
    # print(model.summary())
    # exit()
#     model.compile(loss='categorical_crossentropy',
#                   optimizer='adam',
#                   metrics=['accuracy'])



    model_save_path = 'model/model_weight_attention_st2_{}.h5'.format(str(i))
    if i in [-1]:
        model = model.load_weights(model_save_path)
        print(model.evaluate(X_val, X_val_label))
    else:
        ear = EarlyStopping(monitor='val_loss', min_delta=0, patience=3, verbose=0, mode='min', baseline=None,
                            restore_best_weights=False)

        checkpoint = model_checkpoint = ModelCheckpoint(model_save_path, save_best_only=True, save_weights_only=True)
        history = model.fit(X_train, X_train_label,
                            batch_size=64,
                            epochs=7,
                            shuffle=True,
                            validation_data=(X_val, X_val_label), callbacks=[tensorboard,checkpoint])
        model.load_weights(model_save_path)
        model.save('./model/model_weight_attention_st2_{}.h5'.format(str(i)))

        #model = load_model('model_weight.h5')
    pred_val = model.predict(X_val)
    m_log_loss = log_loss(pred_val, X_val_label)
    print(m_log_loss)
    
    
    
    pred_test = model.predict(x_out_padded_seqs)

    meta_train[te_ind] = pred_val
    meta_test += pred_test
    K.clear_session()
     
meta_test /= 5.0
with open("attention_st2_result.pkl", 'wb') as f:
    pickle.dump(meta_train, f)
    pickle.dump(meta_test, f)




FOLD: 0
2780 11107
Train on 11107 samples, validate on 2780 samples
Epoch 1/7
11107/11107 [==============================] - 38s 3ms/step - loss: 1.4265 - acc: 0.4797 - val_loss: 1.1467 - val_acc: 0.5802
Epoch 2/7
11107/11107 [==============================] - 37s 3ms/step - loss: 0.9442 - acc: 0.6698 - val_loss: 0.8188 - val_acc: 0.7345
Epoch 3/7
11107/11107 [==============================] - 37s 3ms/step - loss: 0.7623 - acc: 0.7438 - val_loss: 0.7415 - val_acc: 0.7622
Epoch 4/7
11107/11107 [==============================] - 37s 3ms/step - loss: 0.6860 - acc: 0.7682 - val_loss: 0.6888 - val_acc: 0.7820
Epoch 5/7
11107/11107 [==============================] - 37s 3ms/step - loss: 0.6374 - acc: 0.7857 - val_loss: 0.6594 - val_acc: 0.7888
Epoch 6/7
11107/11107 [==============================] - 37s 3ms/step - loss: 0.6012 - acc: 0.7969 - val_loss: 0.6647 - val_acc: 0.7871
Epoch 7/7
11107/11107 [==============================] - 37s 3ms/step - loss: 0.5736 - acc: 0.8052 - val_loss: 0.636

ValueError: Multioutput target data is not supported with label binarization

In [24]:
inf = pickle.load(open("dpcnn_st3_result.pkl",encoding='utf-8'))

UnicodeDecodeError: 'utf-8' codec can't decode byte 0x80 in position 0: invalid start byte

In [27]:
out = []
result = meta_test
for i in range(len(file_names)):
    tmp = []
    a = result[i].tolist()

    tmp.append(file_names[i])
    tmp.extend(a)
    out.append(tmp)
with open("result_dpcnn_st3.csv", "w", newline='') as csvfile:
    writer = csv.writer(csvfile)

    # 先写入columns_name
    writer.writerow(["file_id", "prob0", "prob1", "prob2", "prob3", "prob4", "prob5", "prob6", "prob7"])
    # 写入多行用writerows
    writer.writerows(out)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]

In [60]:
class RocAucEvaluation(Callback):
    def __init__(self, validation_data=(), interval=1):
        super(Callback, self).__init__()

        self.interval = interval
        self.X_val, self.y_val = validation_data

    def on_epoch_end(self, epoch, logs={}):
        if epoch % self.interval == 0:
            y_pred = self.model.predict(self.X_val, verbose=0)
            score = roc_auc_score(self.y_val, y_pred)
            print("\n ROC-AUC - epoch: %d - score: %.6f \n" % (epoch+1, score))

def schedule(ind):
    a = [0.001, 0.0005, 0.0001, 0.0001]
    return a[ind] 


batch_size = 128
epochs = 8
maxlen = 1000
x_train_seq = pad_sequences(x_train_word_ids, maxlen=maxlen)

Xtrain, Xval, ytrain, yval = train_test_split(x_train_seq, labels_d, train_size=0.95, random_state=233)

lr = callbacks.LearningRateScheduler(schedule)
ra_val = RocAucEvaluation(validation_data=(Xval, yval), interval = 1)
model.fit(Xtrain, ytrain, batch_size=batch_size, epochs=epochs, validation_data=(Xval, yval), callbacks = [lr, ra_val] ,verbose=1)

Train on 13192 samples, validate on 695 samples
Epoch 1/8
13192/13192 [==============================] - 50s 4ms/step - loss: -21.6155 - acc: 0.2216 - val_loss: -32.6324 - val_acc: 0.1683


ValueError: multiclass format is not supported